In [27]:
"""
    @organization: Fermilab
    @author: Daniel Zurawski
    @author: Keshav Kapoor
    
    Created on Fri June 16 10:43:05 2017
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt








# from keras.models import Sequential
# from keras.layers import Dense, Activation
# from keras.callbacks import EarlyStopping, ModelCheckpoint
#     simple = Sequential()
#     simple.add(Dense(32, input_shape= (1, train_data[0].shape[0]), activation='relu'))
#     simple.add(Dense(train_target.size, kernel_initializer='uniform'))
#     simple.add(Activation('softmax'))
#     simple.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
#      
#     simple.summary()
#      
#     hist = simple.fit( [train_data],
#                       [train_target],
#                       epochs=10,
#                       batch_size=5,
#                       verbose=1,
#                       validation_split=0.2,
#                       callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min'),
#                                 ModelCheckpoint(filepath='simple.h5', verbose=0)]
#                       )
#     show_losses([("cat x entropy", hist)])



In [28]:
def show_losses( histories ):
    plt.figure(figsize=(10,10))
    #plt.ylim(bottom=0)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Error by Epoch')
    colors=[]
    do_acc=False
    for label,loss in histories:
        color = tuple(np.random.random(3))
        colors.append(color)
        l = label
        vl= label+" validation"
        if 'acc' in loss.history:
            l+=' (acc %2.4f)'% (loss.history['acc'][-1])
            do_acc = True
        if 'val_acc' in loss.history:
            vl+=' (acc %2.4f)'% (loss.history['val_acc'][-1])
            do_acc = True
        plt.plot(loss.history['loss'], label=l, color=color)
        if 'val_loss' in loss.history:
            plt.plot(loss.history['val_loss'], lw=2, ls='dashed', label=vl, color=color)


    plt.legend()
    plt.yscale('log')
    plt.show()
    if not do_acc: return
    plt.figure(figsize=(10,10))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    for i,(label,loss) in enumerate(histories):
        color = colors[i]
        if 'acc' in loss.history:
            plt.plot(loss.history['acc'], lw=2, label=label+" accuracy", color=color)
        if 'val_acc' in loss.history:
            plt.plot(loss.history['val_acc'], lw=2, ls='dashed', label=label+" validation accuracy", color=color)
    plt.legend(loc='lower right')
    plt.show()

In [62]:
def get_input_data(dataframe):
    """ Retrieve the input data in the correct format from 'dataframe'.
        @param  dataframe :: pd.DataFrame
                    A data frame with headers: (id, z, phi, eta, val, r, zl).
        @return np.array 2-D
                    A matrix with columns of data: (r, phi, z)
                    
    """
    return dataframe[["r", "phi", "z"]]

In [63]:
def get_output_data(dataframe):
    """ Retrieve the output data in the correct format form 'dataframe'.
        @param  dataframe :: pd.DataFrame
                    A data frame with headers: (id, z, phi, eta, val, r, zl).
        @return np.array 2-D
                    A matrix where a given column at index i is the probability
                    vector of which track the particle at index i belongs to.
    """
    IDs     = dataframe[["id"]].values[:,0] # The track id's for each hit.
    uniques = np.unique(IDs) # The unique track id's.
    matrix  = np.zeros((IDs.size, uniques.size)) # The probability matrix.

    # Create a way to map track id number to an index in the probability matrix.
    ID2row = dict() 
    for col, ID in enumerate(uniques):
        ID2row[ID] = col
     
    # Populate the probability matrix with 100% certainty values.
    for col, ID in enumerate(IDs):
        matrix[col, ID2row[ID]] = 1

    return matrix


In [80]:
filename  = ('file_o_stuff3.csv')
dataframe = pd.read_csv(filename)
train_data   = [get_input_data (dataframe)]
train_target = [get_output_data(dataframe)]
print(train_data[0])
print(train_target[0])
   
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
     
simple = Sequential()
simple.add(Dense(32, input_dim=3, activation='relu'))
simple.add(Dense(9, kernel_initializer='uniform'))
simple.add(Activation('softmax'))
simple.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

simple.summary()
       
   

      r       phi           z
0    40  2.000826   18.712032
1    40  3.674874  -36.069405
2    40  3.164030 -119.277446
3    40  1.850074  -98.647221
4    40  0.706483 -147.321245
5    40  0.446970  302.068300
6    40  3.101934   72.622096
7    70  2.000826   18.712032
8    70  3.674874  -36.069405
9    70  3.164030 -119.277446
10   70  0.446970  302.068300
11   70  3.101934   72.622096
12  100  2.000826   18.712032
13  100  3.674874  -36.069405
14  100  3.164030 -119.277446
15  100  5.341990  263.497157
16  100  3.208663 -470.047981
17  100  0.446970  302.068300
18  100  3.101934   72.622096
19  130  2.000826   18.712032
20  130  3.674874  -36.069405
21  130  3.164030 -119.277446
22  130  5.341990  263.497157
23  130  3.208663 -470.047981
24  130  3.101934   72.622096
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [79]:
 hist = simple.fit( train_data,
                       train_target,
                       epochs=10,
                       batch_size=1,
                       verbose=1,
                       validation_split=0.2)


Train on 20 samples, validate on 5 samples
Epoch 1/10


KeyError: '[8] not in index'

In [51]:
show_losses([("cat x entropy", hist)])

NameError: name 'hist' is not defined